In [1]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import math

class VehicleTracker:
    def __init__(self, video_path, model_path="yolo11n.pt"):
        # โหลดโมเดล YOLOv11
        self.model = YOLO(model_path)
        
        # ตั้งค่าวิดีโอ
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        
        if not self.cap.isOpened():
            print("❌ ไม่สามารถเปิดไฟล์วิดีโอ ตรวจสอบว่าไฟล์อยู่ที่ตำแหน่งที่ถูกต้อง")
            exit()
        
        # ข้อมูลวิดีโอ
        self.fps = self.cap.get(cv2.CAP_PROP_FPS) or 30
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # ตั้งค่าการติดตามและนับ
        self.vehicle_classes = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}
        self.tracked_vehicles = {}
        self.vehicle_trajectories = defaultdict(deque)  # เก็บประวัติการเคลื่อนที่
        self.vehicle_id_counter = 1
        self.crossed_vehicles = defaultdict(int)
        
        # ตั้งค่าพารามิเตอร์
        self.meters_per_pixel = 0.02  # ปรับตามความเป็นจริง
        self.motion_threshold = 2.0  # km/h
        self.max_trajectory_length = 30  # จำนวนเฟรมที่เก็บประวัติ
        self.max_disappeared_frames = 10  # จำนวนเฟรมที่ยอมให้หายไป
        
        # ตั้งค่าเส้นนับรถ (กำหนดพิกัด y สำหรับเส้นนับ)
        self.counting_line_y = self.frame_height // 4
        self.counting_zone_height = 50  # ความสูงของโซนนับ
        
        # ตัวแปรสำหรับ Optical Flow
        self.prev_gray = None
        
        # ตัวแปรสำหรับการนับที่แม่นยำ
        self.vehicle_counted = set()  # เก็บ ID ของรถที่นับแล้ว
        
    def calculate_distance(self, point1, point2):
        """คำนวณระยะทางระหว่างจุด"""
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    
    def predict_direction(self, trajectory):
        """ทำนายทิศทางการเคลื่อนที่จากประวัติการเคลื่อนที่"""
        if len(trajectory) < 3:
            return None, 0
        
        # ใช้ค่าเฉลี่ยของการเคลื่อนที่ใน 5 เฟรมล่าสุด
        recent_points = list(trajectory)[-5:]
        dx_total = 0
        dy_total = 0
        
        for i in range(1, len(recent_points)):
            dx_total += recent_points[i][0] - recent_points[i-1][0]
            dy_total += recent_points[i][1] - recent_points[i-1][1]
        
        if len(recent_points) > 1:
            avg_dx = dx_total / (len(recent_points) - 1)
            avg_dy = dy_total / (len(recent_points) - 1)
        else:
            avg_dx = avg_dy = 0
        
        # คำนวณมุมทิศทาง
        angle = math.atan2(avg_dy, avg_dx) * 180 / math.pi
        speed = math.sqrt(avg_dx**2 + avg_dy**2)
        
        return angle, speed
    
    def assign_vehicle_id(self, detections):
        """กำหนด ID ให้กับรถที่ตรวจพบ"""
        current_vehicles = {}
        
        for detection in detections:
            x1, y1, x2, y2, cls, conf = detection
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            
            # หา ID ที่ใกล้เคียงที่สุด
            best_match_id = None
            min_distance = float('inf')
            
            for vehicle_id, vehicle_data in self.tracked_vehicles.items():
                if vehicle_data['disappeared_frames'] <= self.max_disappeared_frames:
                    last_center = vehicle_data['last_center']
                    distance = self.calculate_distance(center, last_center)
                    
                    # ตรวจสอบว่าเป็นประเภทรถเดียวกันและระยะทางไม่เกินขีดจำกัด
                    if (distance < min_distance and distance < 100 and 
                        vehicle_data['class'] == cls):
                        min_distance = distance
                        best_match_id = vehicle_id
            
            if best_match_id is not None:
                # อัปเดตข้อมูลรถที่มีอยู่
                vehicle_id = best_match_id
                self.tracked_vehicles[vehicle_id].update({
                    'bbox': (x1, y1, x2, y2),
                    'last_center': center,
                    'disappeared_frames': 0,
                    'confidence': conf
                })
            else:
                # สร้างรถใหม่
                vehicle_id = self.vehicle_id_counter
                self.vehicle_id_counter += 1
                self.tracked_vehicles[vehicle_id] = {
                    'bbox': (x1, y1, x2, y2),
                    'last_center': center,
                    'class': cls,
                    'disappeared_frames': 0,
                    'confidence': conf,
                    'first_seen_frame': True
                }
            
            # เพิ่มจุดใหม่ในประวัติการเคลื่อนที่
            self.vehicle_trajectories[vehicle_id].append(center)
            if len(self.vehicle_trajectories[vehicle_id]) > self.max_trajectory_length:
                self.vehicle_trajectories[vehicle_id].popleft()
            
            current_vehicles[vehicle_id] = self.tracked_vehicles[vehicle_id]
        
        # เพิ่มจำนวนเฟรมที่หายไปสำหรับรถที่ไม่พบ
        for vehicle_id in list(self.tracked_vehicles.keys()):
            if vehicle_id not in current_vehicles:
                self.tracked_vehicles[vehicle_id]['disappeared_frames'] += 1
                if self.tracked_vehicles[vehicle_id]['disappeared_frames'] > self.max_disappeared_frames:
                    del self.tracked_vehicles[vehicle_id]
                    if vehicle_id in self.vehicle_trajectories:
                        del self.vehicle_trajectories[vehicle_id]
        
        return current_vehicles
    
    def check_line_crossing(self, vehicle_id, current_center, previous_center):
        """ตรวจสอบการข้ามเส้นนับรถ"""
        line_y = self.counting_line_y
        zone_top = line_y - self.counting_zone_height // 2
        zone_bottom = line_y + self.counting_zone_height // 2
        
        # ตรวจสอบการข้ามเส้นจากบนลงล่าง หรือ ล่างขึ้นบน
        if previous_center is not None:
            prev_y = previous_center[1]
            curr_y = current_center[1]
            
            # ตรวจสอบว่ารถอยู่ในโซนนับและยังไม่เคยนับ
            if (zone_top <= curr_y <= zone_bottom and 
                vehicle_id not in self.vehicle_counted):
                
                # ตรวจสอบทิศทางการข้าม
                if prev_y < zone_top and curr_y >= zone_top:
                    # ข้ามจากบนลงล่าง
                    return "down"
                elif prev_y > zone_bottom and curr_y <= zone_bottom:
                    # ข้ามจากล่างขึ้นบน
                    return "up"
        
        return None
    
    def calculate_speed_optical_flow(self, frame, prev_frame, bbox):
        """คำนวณความเร็วด้วย Optical Flow"""
        x1, y1, x2, y2 = bbox
        
        # ตรวจสอบขอบเขต
        if x1 < 0 or y1 < 0 or x2 >= frame.shape[1] or y2 >= frame.shape[0]:
            return 0
        
        # แปลงเป็น grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        
        # ดึง ROI
        roi = gray[y1:y2, x1:x2]
        prev_roi = prev_gray[y1:y2, x1:x2]
        
        if roi.size == 0 or prev_roi.size == 0:
            return 0
        
        # คำนวณ Optical Flow
        flow = cv2.calcOpticalFlowFarneback(
            prev_roi, roi, None, 0.5, 3, 15, 3, 5, 1.2, 0
        )
        
        # คำนวณความเร็วเฉลี่ย
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        speed_px_per_frame = np.mean(magnitude)
        
        # แปลงเป็น km/h
        speed_m_per_s = speed_px_per_frame * self.meters_per_pixel * self.fps
        speed_km_per_h = speed_m_per_s * 3.6
        
        return speed_km_per_h
    
    def draw_counting_line(self, frame):
        """วาดเส้นนับรถ"""
        line_y = self.counting_line_y
        zone_top = line_y - self.counting_zone_height // 2
        zone_bottom = line_y + self.counting_zone_height // 2
        
        # วาดโซนนับ (สีเหลืองโปร่งใส)
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, zone_top), (self.frame_width, zone_bottom), 
                     (0, 255, 255), -1)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
        
        # วาดเส้นกลาง
        cv2.line(frame, (0, line_y), (self.frame_width, line_y), (0, 0, 255), 3)
        cv2.putText(frame, "COUNTING LINE", (10, line_y - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    def process_frame(self, frame):
        """ประมวลผลเฟรม"""
        # ตรวจจับรถ
        results = self.model(frame, classes=[2, 3, 5, 7])  # รถยนต์, มอเตอร์ไซค์, รถบัส, รถบรรทุก
        
        detections = []
        if results[0].boxes is not None:
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = box.conf[0].cpu().numpy()
                cls = int(box.cls[0].cpu().numpy())
                
                if conf > 0.5:  # เฉพาะการตรวจจับที่มีความเชื่อมั่นสูง
                    detections.append((x1, y1, x2, y2, cls, conf))
        
        # กำหนด ID และติดตามรถ
        current_vehicles = self.assign_vehicle_id(detections)
        
        # วาดเส้นนับรถ
        self.draw_counting_line(frame)
        
        # ประมวลผลแต่ละรถ
        for vehicle_id, vehicle_data in current_vehicles.items():
            x1, y1, x2, y2 = vehicle_data['bbox']
            cls = vehicle_data['class']
            conf = vehicle_data['confidence']
            current_center = vehicle_data['last_center']
            
            # ดึงข้อมูลประวัติการเคลื่อนที่
            trajectory = self.vehicle_trajectories[vehicle_id]
            
            # ทำนายทิศทางและความเร็ว
            direction_angle, trajectory_speed = self.predict_direction(trajectory)
            
            # คำนวณความเร็วด้วย Optical Flow (ถ้ามีเฟรมก่อนหน้า)
            optical_flow_speed = 0
            if self.prev_gray is not None:
                optical_flow_speed = self.calculate_speed_optical_flow(
                    frame, self.prev_gray, (x1, y1, x2, y2)
                )
            
            # ใช้ความเร็วที่แม่นยำกว่า
            final_speed = max(trajectory_speed * self.meters_per_pixel * self.fps * 3.6, 
                            optical_flow_speed)
            
            # ตรวจสอบการข้ามเส้นนับ
            if len(trajectory) >= 2:
                previous_center = trajectory[-2] if len(trajectory) >= 2 else None
                crossing_direction = self.check_line_crossing(
                    vehicle_id, current_center, previous_center
                )
                
                if crossing_direction and vehicle_id not in self.vehicle_counted:
                    vehicle_type = self.vehicle_classes.get(cls, "Unknown")
                    self.crossed_vehicles[vehicle_type] += 1
                    self.vehicle_counted.add(vehicle_id)
            
            # วาดกรอบและข้อมูล
            color = (0, 255, 0) if final_speed > self.motion_threshold else (0, 165, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # แสดงข้อมูลรถ
            vehicle_type = self.vehicle_classes.get(cls, "Unknown")
            info_text = f"ID:{vehicle_id} {vehicle_type}"
            cv2.putText(frame, info_text, (x1, y1 - 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # แสดงความเร็ว
            speed_text = f"Speed: {final_speed:.1f} km/h"
            cv2.putText(frame, speed_text, (x1, y1 - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # แสดงความเชื่อมั่น
            conf_text = f"Conf: {conf:.2f}"
            cv2.putText(frame, conf_text, (x1, y1 - 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
            
            # วาดเส้นทิศทาง
            if direction_angle is not None and final_speed > self.motion_threshold:
                arrow_length = 50
                end_x = int(current_center[0] + arrow_length * math.cos(math.radians(direction_angle)))
                end_y = int(current_center[1] + arrow_length * math.sin(math.radians(direction_angle)))
                cv2.arrowedLine(frame, current_center, (end_x, end_y), (0, 0, 255), 3)
            
            # วาดเส้นทางการเคลื่อนที่
            if len(trajectory) > 1:
                points = np.array(list(trajectory), np.int32)
                cv2.polylines(frame, [points], False, (255, 0, 0), 2)
        
        # แสดงสถิติการนับ
        y_offset = 30
        total_vehicles = sum(self.crossed_vehicles.values())
        cv2.putText(frame, f"Total Vehicles: {total_vehicles}", 
                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        y_offset += 30
        for vehicle_type, count in self.crossed_vehicles.items():
            cv2.putText(frame, f"{vehicle_type}: {count}", 
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            y_offset += 25
        
        # แสดงจำนวนรถที่ติดตามอยู่
        cv2.putText(frame, f"Tracking: {len(current_vehicles)}", 
                   (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        return frame
    
    def run(self):
        """เรียกใช้ระบบติดตามรถ"""
        print("🚗 เริ่มระบบติดตามและนับรถ YOLOv11")
        print("กด 'q' เพื่อออกจากโปรแกรม")
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # ประมวลผลเฟรม
            processed_frame = self.process_frame(frame)
            
            # เก็บเฟรมปัจจุบันสำหรับ Optical Flow
            self.prev_gray = frame.copy()
            
            # แสดงผล
            cv2.imshow("YOLOv11 Vehicle Detection & Tracking", processed_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # แสดงสถิติสุดท้าย
        print("\n📊 สถิติการนับรถ:")
        print(f"รวมทั้งหมด: {sum(self.crossed_vehicles.values())} คัน")
        for vehicle_type, count in self.crossed_vehicles.items():
            print(f"{vehicle_type}: {count} คัน")
        
        self.cap.release()
        cv2.destroyAllWindows()

# การใช้งาน
if __name__ == "__main__":
    # 🔥 ใส่ path ของไฟล์วิดีโอที่ต้องการเทส
    video_path = "Traffic IP Camera video.mp4"
    
    # สร้างและเรียกใช้ระบบ
    tracker = VehicleTracker(video_path, model_path="yolo11n.pt")
    tracker.run()

ERROR Error reading from C:\Users\earth\AppData\Roaming\Ultralytics\settings.json: "No Ultralytics setting 'openvino_msg'. \nView Ultralytics Settings with 'yolo settings' or at 'C:\\Users\\earth\\AppData\\Roaming\\Ultralytics\\settings.json'\nUpdate Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings."
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\earth\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚗 เริ่มระบบติดตามและนับรถ YOLOv11
กด 'q' เพื่อออกจากโปรแกรม

0: 384x640 2 cars, 50.8ms
Speed: 4.2ms preprocess, 50.8ms inference, 124.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.4ms
Speed: 1.2ms preprocess, 5.4ms inference, 1.4ms p